In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import csv
import numpy
# You may or may not want to use this package, or others like it
# this is just a starting point for you
from sklearn.linear_model import LinearRegression

import pandas as pd
import os

In [3]:
# Read the player database into an array of dictionaries
players = []
with open('playerDB.csv', mode='r') as player_csv:
    player_reader = csv.DictReader(player_csv)
    line_count = 0
    for row in player_reader:
        players.append(dict(row))

# Read the draft database into an array of dictionaries
draftPicks = []
with open('draftDB.csv', mode='r') as draft_csv:
    draft_reader = csv.DictReader(draft_csv)
    line_count = 0
    for row in draft_reader:
        draftPicks.append(dict(row))


In [115]:
players_df = pd.read_csv('playerDB.csv')
draft_df = pd.read_csv('draftDB.csv')

link_df = pd.read_csv('NBA_Player_IDs.csv',  encoding= 'unicode_escape')
link_df = link_df[pd.notna(link_df['NBAID'])]
link_df.loc[:,'NBAID'] = link_df.loc[:,'NBAID'].astype(int)

draft_df = draft_df[draft_df['numberRound']<=2]
draft_df = draft_df[draft_df['yearDraft']>=1979]
draft_df = draft_df[draft_df['yearDraft']<=2015]

#draft_df.loc[:,'urlID'] = draft_df['basketball_reference_url'].apply(lambda url: os.path.splitext(os.path.basename(str(url)))[0])
players_df.loc[:,'year'] = players_df['Season'].apply(lambda season: int(season.split('-')[0]))

curr_miss = draft_df[draft_df.basketball_reference_url.isna()]
print(f'Missing {len(curr_miss)} links between player and draft')

draft_df = draft_df.merge(link_df, how='left', left_on='idPlayer', right_on='NBAID')
draft_df.loc[:,'urlID'] = draft_df.loc[:,'BBRefID'] 

curr_miss = draft_df[draft_df.urlID.isna()]
print(f'Missing {len(curr_miss)} links between player and draft')

combined_df = players_df.merge(draft_df, how='inner', on='urlID')

num_players_yrs = len(players_df)
num_players = len(players_df.urlID.unique())

print(f'{num_players} distinct players with {num_players_yrs} player-seasons of data')

Missing 684 links between player and draft
Missing 297 links between player and draft
3088 distinct players with 16614 player-seasons of data


297

Missing 2028 links between player and draft


In [113]:
players_df

,Season,Rk,Player,urlID,Pos,Age,Tm,G,MP,PER,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,year
0,1979-80,1,Kareem Abdul-Jabbar*,abdulka01,C,32,LAL,82,3143,25.3,...,24.1,9.5,5.3,14.8,0.227,4.0,2.7,6.7,6.8,1979
1,1979-80,2,Tom Abernethy,abernto01,PF,25,GSW,67,1222,11.0,...,13.3,1.2,0.8,2.0,0.080,-1.5,-0.1,-1.6,0.1,1979
2,1979-80,3,Alvan Adams,adamsal01,C,25,PHO,75,2168,19.2,...,21.9,3.1,3.9,7.0,0.155,1.6,2.8,4.4,3.5,1979
3,1979-80,4,Tiny Archibald*,architi01,PG,31,BOS,80,2864,15.3,...,17.0,5.9,2.9,8.9,0.148,1.1,-1.1,0.0,1.5,1979
4,1979-80,5,Dennis Awtrey,awtrede01,C,31,CHI,26,560,7.4,...,7.9,0.1,0.5,0.6,0.053,-2.9,1.5,-1.4,0.1,1979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16609,2018-19,526,Trae Young,youngtr01,PG,20,ATL,81,2503,17.0,...,28.4,2.5,0.7,3.3,0.062,1.8,-2.8,-1.1,0.6,2018
16610,2018-19,527,Cody Zeller,zelleco01,C,26,CHO,49,1243,17.2,...,16.3,2.6,1.2,3.9,0.150,0.2,1.7,2.0,1.2,2018
16611,2018-19,528,Tyler Zeller,zellety01,C,29,TOT,6,93,17.3,...,20.1,0.2,0.1,0.3,0.167,-1.0,-1.2,-2.2,0.0,2018
16612,2018-19,529,Ante _i_i?,zizican01,C,22,CLE,59,1082,16.2,...,18.2,1.7,0.3,2.0,0.087,-1.7,-1.4,-3.1,-0.3,2018


In [81]:
missing_draft_ids = draft_df[draft_df['basketball_reference_url'].isna()].idPlayer.unique()
link_ids = link_df.NBAID.unique()
still_missing = [draft_id for draft_id in missing_draft_ids if draft_id not in link_ids]

In [82]:
still_missing_df = draft_df[draft_df['idPlayer'].isin(still_missing)]

In [83]:
still_missing_df

,yearDraft,numberPickOverall,numberRound,numberRoundPick,namePlayer,slugTeam,nameOrganizationFrom,typeOrganizationFrom,idPlayer,idTeam,nameTeam,basketball_reference_url,urlID
265,2015,26,1,26,Nikola Milutinov,SAS,KK Partizan,Other Team/Club,1626221,1610612759,San Antonio Spurs,NaN,nan
278,2015,39,2,9,Juan Pablo Vaulet,CHA,Bahia Basket,Other Team/Club,1626225,1610612766,Charlotte Hornets,NaN,nan
281,2015,42,2,12,Olivier Hanlan,UTA,Boston College,College/University,1626152,1610612762,Utah Jazz,NaN,nan
284,2015,45,2,15,Marcus Thornton,BOS,William & Mary,College/University,1626194,1610612738,Boston Celtics,NaN,nan
286,2015,47,2,17,Arturas Gudaitis,PHI,BC Zalgiris,Other Team/Club,1626222,1610612755,Philadelphia 76ers,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3515,1979,38,2,16,Larry Wilson,ATL,Nicholls State,College/University,82364,1610612737,Atlanta Hawks,NaN,nan
3516,1979,39,2,17,Victor King,LAL,Louisiana Tech,College/University,82342,1610612747,Los Angeles Lakers,NaN,nan
3517,1979,40,2,18,Andrew Fields,POR,Cheyney,College/University,82440,1610612757,Portland Trail Blazers,NaN,nan
3518,1979,41,2,19,Mark Young,LAL,Fairfield,College/University,82345,1610612747,Los Angeles Lakers,NaN,nan


In [75]:
still_missing_df.shape

(28, 13)

In [66]:
302/2268

0.13315696649029982

In [98]:
players_df[players_df['Player']=='Olivier Hanlan']

,Season,Rk,Player,urlID,Pos,Age,Tm,G,MP,PER,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,year


In [84]:
link_df

,BBRefName,BBRefLink,BBRefID,BBRefBirthDate,NBAName,NBALink,NBAID,NBABirthDate,ESPNName,ESPNLink,ESPNID,ESPNBirthDate,SpotracName,SpotracLink,SpotracID
0,A.J. Hammons,https://www.basketball-reference.com/players/h...,hammoaj01,8/27/1992,AJ Hammons,https://stats.nba.com/player/1627773/,1627773,8/27/1992,AJ Hammons,http://www.espn.com/nba/player/_/id/2991178/aj...,2991178.0,8/27/1992,A.J. Hammons,https://www.spotrac.com/redirect/player/20252/,20252.0
1,A.J. Price,https://www.basketball-reference.com/players/p...,priceaj01,10/7/1986,AJ Price,https://stats.nba.com/player/201985/,201985,10/7/1986,A.J. Price,http://www.espn.com/nba/player/_/id/4010/aj-price,4010.0,10/7/1986,A.J. Price,https://www.spotrac.com/redirect/player/6292/,6292.0
2,Aaron Brooks,https://www.basketball-reference.com/players/b...,brookaa01,1/14/1985,Aaron Brooks,https://stats.nba.com/player/201166/,201166,1/14/1985,Aaron Brooks,http://www.espn.com/nba/player/_/id/3192/aaron...,3192.0,1/14/1985,Aaron Brooks,https://www.spotrac.com/redirect/player/2390/,2390.0
3,Aaron Gordon,https://www.basketball-reference.com/players/g...,gordoaa01,9/16/1995,Aaron Gordon,https://stats.nba.com/player/203932/,203932,9/16/1995,Aaron Gordon,http://www.espn.com/nba/player/_/id/3064290/aa...,3064290.0,9/16/1995,Aaron Gordon,https://www.spotrac.com/redirect/player/15356/,15356.0
4,Aaron Gray,https://www.basketball-reference.com/players/g...,grayaa01,12/7/1984,Aaron Gray,https://stats.nba.com/player/201189/,201189,12/7/1984,Aaron Gray,http://www.espn.com/nba/player/_/id/3207/aaron...,3207.0,12/7/1984,Aaron Gray,https://www.spotrac.com/redirect/player/2244/,2244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4524,NaN,NaN,NaN,NaN,Terrico White,https://stats.nba.com/player/202358/,202358,3/7/1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4525,NaN,NaN,NaN,NaN,Lindell Wigginton,https://stats.nba.com/player/1629623/,1629623,3/28/1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4526,NaN,NaN,NaN,NaN,Ward Williams,https://stats.nba.com/player/78573/,78573,6/26/1923,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4527,NaN,NaN,NaN,NaN,Hank Zeller,https://stats.nba.com/player/78642/,78642,7/10/1918,NaN,NaN,NaN,NaN,NaN,NaN,NaN
